# Game of Life

## Required imports

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

## World representation

A Game of Life world will be represented by an array of integers.  Each array element represents a cell that can either be dead (0) or alive (1).  First, define a class that represents a randomly initialized world of $n \times n$, where a fraction $f_{\rm alive}$ is alive and defines methods to compute the next generation and provide a string representation.

In [88]:
class RandomWorld:
    
    def __init__(self, n, f_alive):
        self._tmp_world = np.empty((n, n), dtype=np.int8)
        self._world = np.random.choice(np.array([0, 1], dtype=np.int8), (self.n, self.n),
                                       p=(1 - f_alive, f_alive))
    
    @property
    def n(self):
        return self._tmp_world.shape[0]
    
    @property
    def nr_alive(self):
        return np.sum(self._world)

    @property
    def cells(self):
        return np.copy(self._world)

    @property
    def fraction_alive(self):
        return np.sum(self._world)/self.n**2
    
    def is_alive(self, i, j):
        return self._world[i, j] == 1
    
    def nr_neignbours(self, i, j):
        return np.sum(self._world[i-1:i+2, j-1:j+2]) - self._world[i, j]
    
    def next_generation(self):
        for i in range(self.n):
            for j in range(self.n):
                nr_nb = self.nr_neignbours(i, j)
                if self.is_alive(i, j):
                    self._tmp_world[i, j] = 1 if nr_nb == 2 or nr_nb == 3 else 0
                else:
                    self._tmp_world[i, j] = 1 if nr_nb == 3 else 0
        self._world = self._tmp_world


        
    def __str__(self):
        return '\n'.join(' '.join(f'{self._world[i, j]:1d}' for j in range(self.n)) for i in range(self.n))

Create a world and run a generation.

In [89]:
world = RandomWorld(10, 0.4)

In [90]:
print(world)

0 0 1 1 0 1 1 0 1 1
0 0 1 1 0 1 1 0 0 0
1 1 1 0 0 1 1 0 0 0
0 1 0 1 0 0 1 0 0 1
0 0 0 0 0 0 0 1 0 0
1 0 0 1 0 0 1 0 0 1
1 1 0 1 1 1 0 0 1 1
1 0 1 0 0 0 0 0 1 1
1 1 1 0 1 0 1 1 0 0
1 0 0 1 0 0 1 0 0 0


In [91]:
world.next_generation()

In [92]:
print(world)

0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 1 0 0
0 1 0 0 0 1 1 1 0 0
0 0 1 0 0 0 1 1 1 0
0 1 1 1 0 1 1 1 0 1
0 0 0 1 1 1 0 1 0 0
0 0 0 0 0 0 1 0 0 1
0 0 1 0 0 1 1 1 1 0
0 0 1 1 0 1 1 1 0 0


## Simulation runner

We define a class to conveniently perform a complete simulation.  At most `max_gen` generations are computed, but the computation stops as soon as a cycle is detected.

In [93]:
class WorldRunner:
    
    def __init__(self, world, max_gen):
        self._world = world
        self._max_gen = max_gen
        self._cycle_length = None
        self._hist = [self._world.cells]
    
    @property
    def max_gen(self):
        return self._max_gen

    @property
    def generation(self):
        return len(self._hist) - 1
    
    def has_cycle(self):
        return self._cycle_length is not None
    
    @property
    def cycle_length(self):
        return self._cycle_length
    
    @property
    def history(self):
        return self._hist
    
    def _has_cycle(self):
        for gen in range(-2, -len(self._hist), -1):
            if np.all(self._hist[-1] == self._hist[gen]):
                self._cycle_length = -gen - 1
                return True
        return False
    
    def run(self):
        for _ in range(1, self.max_gen + 1):
            self._world.next_generation()
            self._hist.append(self._world.cells)
            if self._has_cycle():
                break

Create a world, and run it for a number of generations, then check on the properties.

In [94]:
world = RandomWorld(10, 0.3)
runner = WorldRunner(world, 100)
runner.run()

The current state of the world can be checked.

In [95]:
print(world)

0 0 0 0 0 0 0 0 0 0
0 0 0 0 1 1 0 0 0 0
0 0 0 0 1 1 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 1 1
0 0 0 0 0 0 1 0 0 1
0 0 0 0 0 0 1 1 0 0


In [99]:
world.fraction_alive

0.1

Check whether a cycle has been detected, what the cycle length is, and after how many generations it occured.

In [96]:
runner.has_cycle()

True

In [97]:
runner.cycle_length

1

In [98]:
runner.generation

44